In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn . discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn . discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [17]:
data = pd.read_csv('WMT1.csv')
data_17 = data[data.Year == 2017]

# calculate the mean std
mean_17 = data_17.groupby('Week_Number')['Return'].mean().tolist()
std_17 = data_17.groupby('Week_Number')['Return'].std().tolist()
week_id_17 = list(data_17.groupby('Week_Number').indices.keys())
color_17 = data_17.groupby('Week_Number')['label'].agg(lambda x: x.mode()).tolist()

data_18 = data[data.Year == 2018]
data_18.drop(data_18.index[-1], inplace=True)

# calculate the mean std
mean_18 = data_18.groupby('Week_Number')['Return'].mean().tolist()
std_18 = data_18.groupby('Week_Number')['Return'].std().tolist()
weekly_18 = data[(data.Year == 2018) & (data.Weekday == 'Friday')]['Adj Close'].values  # select Friday's data

week_id_18 = list(data_18.groupby('Week_Number').indices.keys())
color_18 = data_18.groupby('Week_Number')['label'].agg(lambda x: x.mode()).tolist()


/Users/wangxiaoyang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
df_17 = pd.DataFrame(
    {
        'id':week_id_17,
        'mean':mean_17,
        'std':std_17,
        'label':color_17
    },
    columns = ['id','mean','std','label']
)

df_18 = pd.DataFrame(
    {
        'id':week_id_18,
        'mean':mean_18,
        'std':std_18,
        'label':color_18
    },
    columns = ['id','mean','std','label']
)

In [19]:
x_train = df_17[['mean','std']].values
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
y_train= df_17['label'].values

x_test = df_18[['mean','std']].values
scaler = StandardScaler()
scaler.fit(x_test)
x_test = scaler.transform(x_test)
y_test = df_18['label'].values

In [37]:
#question2
lda_classifier = LDA()
lda_classifier.fit(x_train,y_train)
predicted = lda_classifier.predict(x_test)
accuracy = lda_classifier.score (x_test, y_test)
accuracy

0.8461538461538461

In [51]:
#question1
a = lda_classifier.coef_
b = lda_classifier.intercept_
print('y = %f*x1+%f*x2+%f'%(a[0][0],a[0][1],b[0]))

y = -2.605701*x1+0.566258*x2+-1.006331


In [25]:
qda_classifier = QDA()
qda_classifier.fit(x_train,y_train)
predicted1 = qda_classifier.predict(x_test)
accuracy1 = qda_classifier.score (x_test, y_test)
accuracy1

0.9038461538461539

In [26]:
print('QDA has better accuracy')

QDA has better accuracy


In [27]:
#question3

a = confusion_matrix(y_test,predicted)
a

array([[31,  0],
       [ 8, 13]])

In [28]:
a1 = confusion_matrix(y_test,predicted1)
a1

array([[31,  0],
       [ 5, 16]])

In [29]:
#question4

TPR = a[0][0]/(a[0][0]+a[0][1])
TNR = a[1][1]/(a[1][0]+a[1][1])
print('true positive rate is :',TPR)
print('true negative rate is :',TNR)

true positive rate is : 1.0
true negative rate is : 0.6190476190476191


In [30]:
TPR = a1[0][0]/(a1[0][0]+a1[0][1])
TNR = a1[1][1]/(a1[1][0]+a1[1][1])
print('true positive rate is :',TPR)
print('true negative rate is :',TNR)

true positive rate is : 1.0
true negative rate is : 0.7619047619047619


In [31]:
#question5
previous_shares = 0
previous_balance = 100
weekly_balance = []
for i in range(len(weekly_18)-1):
    if predicted[i+1] == 'green':
        if previous_shares ==0 :
            previous_shares = previous_balance/weekly_18[i]
    elif predicted[i+1] == 'red':
        if previous_shares != 0:
            previous_balance = previous_shares*weekly_18[i]
            previous_shares = 0

    weekly_balance.append(previous_balance)
result = weekly_balance[-1]

In [32]:
previous_shares = 0
previous_balance = 100
weekly_balance = []
for i in range(len(weekly_18)-1):
    if predicted1[i+1] == 'green':
        if previous_shares ==0 :
            previous_shares = previous_balance/weekly_18[i]
    elif predicted1[i+1] == 'red':
        if previous_shares != 0:
            previous_balance = previous_shares*weekly_18[i]
            previous_shares = 0

    weekly_balance.append(previous_balance)
result1 = weekly_balance[-1]

In [33]:
result2 = 100/weekly_18[0]*weekly_18[-1]

print('result based on LDA: %f $'%result)
print('result based on QDA: %f $'%result1)
print('result based on buy and hold: %f $'%result2)
print('LDA results in a large amount at the end of the year')

result based on LDA: 116.527373 $
result based on QDA: 115.768661 $
result based on buy and hold: 94.188504 $
buy and hold strategy results in a large amount at the end of the year
